In [10]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import scipy as sp

In [11]:
# SIS simulation for complex network
def sis_simulation(G, beta, mu, initial_infected=0.1, Tmax=1000, Ttrans=200):
    N = G.number_of_nodes()
    infected = np.zeros(N, dtype=bool)
    infected[np.random.choice(N, int(N * initial_infected), replace=False)] = True
    result_timeseries = []

    for _ in range(Tmax):
        new_infected = infected.copy()
        for node in G.nodes:
            if infected[node] and np.random.random() < mu:
                new_infected[node] = False
            elif not infected[node]:
                neighbors = [neighbor for neighbor in G.neighbors(node) if infected[neighbor]]
                if neighbors and np.random.random() < 1 - (1 - beta) ** len(neighbors):
                    new_infected[node] = True
        infected = new_infected
        if _ >= Ttrans:
            result_timeseries.append(infected.sum() / N)
    return np.mean(result_timeseries), result_timeseries

In [12]:
# MMCA for complex network 
def mmca_prediction(G, beta, mu):
    N = G.number_of_nodes()
    A = nx.adjacency_matrix(G).todense()
    p = np.random.rand(N) * 0.1  # Initial infected probabilities as small random values
    convergence_threshold = 1e-4

    for _ in range(1000):
        new_p = beta * (1 - p) * np.dot(A, p) + (1 - mu) * p
        if np.linalg.norm(new_p - p, 1) < convergence_threshold:
            break
        p = new_p
    return np.mean(p)

In [13]:
# Create networks for simulation
def create_networks(N):
    networks = {
        "Erdos_Renyi": nx.erdos_renyi_graph(n=N, p=0.01),
        "Scale_Free": nx.barabasi_albert_graph(n=N, m=5),
    }
    return networks

In [24]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

def initialize_infection(N, initial_infected_ratio=0.1):
    """Initializes the infected population in the network."""
    infected = np.zeros(N, dtype=bool)
    infected[np.random.choice(N, int(N * initial_infected_ratio), replace=False)] = True
    return infected

def sis_simulation(G, beta, mu, Tmax=1000, Ttrans=200):
    """Simulates the SIS model using the Monte Carlo method."""
    N = G.number_of_nodes()
    infected = initialize_infection(N)
    result_timeseries = []

    for _ in range(Tmax):
        new_infected = infected.copy()
        for node in G.nodes:
            if infected[node] and np.random.random() < mu:
                new_infected[node] = False
            elif not infected[node]:
                if any(np.random.random() < beta for neighbor in G.neighbors(node) if infected[neighbor]):
                    new_infected[node] = True
        infected = new_infected
        if _ >= Ttrans:
            result_timeseries.append(np.mean(infected))
    return np.mean(result_timeseries), result_timeseries

def mmca_prediction(G, beta, mu):
    """Calculates the stationary state fraction of infected nodes using the MMCA model."""
    N = G.number_of_nodes()
    A = nx.adjacency_matrix(G).todense()
    p = np.random.rand(N) * 0.1
    for _ in range(1000):
        new_p = beta * (1 - p) * (A @ p) + (1 - mu) * p
        if np.allclose(p, new_p, atol=1e-4):
            break
        p = new_p
    return np.mean(p)

def create_networks(N, k=10, p=0.01):
    """Generates different types of networks based on specified parameters."""
    return {
        "Erdos_Renyi": nx.erdos_renyi_graph(n=N, p=p),
        "Scale_Free": nx.barabasi_albert_graph(n=N, m=k),
        "Small_World": nx.watts_strogatz_graph(n=N, k=k, p=0.1)
    }

def run_simulations():
    """Main function to setup simulations, run them, and output results."""
    network_sizes = [500, 1000, 1500]
    beta_values = np.linspace(0, 1, 99)
    mu_values = [0.1, 0.5, 0.9]

    for N in network_sizes:
        networks = create_networks(N, k=10, p=0.01)
        for name, G in networks.items():
            nx.write_pajek(G, f"{name}_{N}.net")
            for mu in mu_values:
                mc_results, mmca_results = [], []
                for beta in tqdm(beta_values, desc=f"Simulating {name} with N={N}, mu={mu}"):
                    mc_rho, _ = sis_simulation(G, beta, mu)
                    mmca_rho = mmca_prediction(G, beta, mu)
                    mc_results.append(mc_rho)
                    mmca_results.append(mmca_rho)

                df = pd.DataFrame({
                    'Beta': beta_values,
                    'MC_Rho': mc_results,
                    'MMCA_Rho': mmca_results
                })
                df.to_csv(f"{name}_N{N}_mu_{mu}.csv", index=False)

                plt.figure(figsize=(10, 6))
                plt.plot(beta_values, mc_results, label='Monte Carlo')
                plt.plot(beta_values, mmca_results, label='MMCA', linestyle='--')
                plt.title(f'SIS Model on {name} Network (N={N}, μ={mu})')
                plt.xlabel('Beta (Infection Probability)')
                plt.ylabel('Rho (Fraction of Infected Nodes)')
                plt.legend()
                plt.grid(True)
                plt.savefig(f"Comparison_{name}_N{N}_mu_{mu}.png")
                plt.close()

if __name__ == '__main__':
    run_simulations()


Simulating Erdos_Renyi with N=500, mu=0.1:  17%|█▋        | 17/99 [00:07<00:33,  2.47it/s]/var/folders/hv/8r0914b93k5__tgxzywr4jf80000gn/T/ipykernel_26192/3322112052.py:38: RuntimeWarning: overflow encountered in multiply
  new_p = beta * (1 - p) * (A @ p) + (1 - mu) * p
/var/folders/hv/8r0914b93k5__tgxzywr4jf80000gn/T/ipykernel_26192/3322112052.py:38: RuntimeWarning: invalid value encountered in matmul
  new_p = beta * (1 - p) * (A @ p) + (1 - mu) * p
Simulating Erdos_Renyi with N=500, mu=0.5:  86%|████████▌ | 85/99 [00:54<00:09,  1.54it/s]/var/folders/hv/8r0914b93k5__tgxzywr4jf80000gn/T/ipykernel_26192/3322112052.py:38: RuntimeWarning: overflow encountered in matmul
  new_p = beta * (1 - p) * (A @ p) + (1 - mu) * p
Simulating Small_World with N=1500, mu=0.9: 100%|██████████| 99/99 [05:25<00:00,  3.29s/it]
